In [1]:
import ee 
import folium

In [2]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the custom method to the folium Map class.
folium.Map.add_ee_layer = add_ee_layer


In [4]:
# Initialize the Earth Engine module.
ee.Initialize()

# Define the start and end date.
START = '2010-04-02'
END = '2018-04-03'

# Define the region of interest.
region = ee.Geometry.Point(14.0, 37.5)

# Filter the collections by region and date.
colFilter = ee.Filter.And(
    ee.Filter.bounds(region),
    ee.Filter.date(START, END))

dwCol = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(colFilter)
s2Col = ee.ImageCollection('COPERNICUS/S2').filter(colFilter)

# Ensure collections are not empty
dwCol_size = dwCol.size().getInfo()
s2Col_size = s2Col.size().getInfo()

if dwCol_size == 0 or s2Col_size == 0:
    raise ValueError("One or both of the collections are empty. Please adjust your filters.")

# Join corresponding DW and S2 images (by system:index).
DwS2Col = ee.Join.saveFirst('s2_img').apply(dwCol, s2Col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'))

# Extract an example DW image and its source S2 image.
dwImage = ee.Image(DwS2Col.first())
s2Image = ee.Image(dwImage.get('s2_img'))

# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']

VIS_PALETTE = [
    '419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b',
    'a59b8f', 'b39fe1']

# Create an RGB image of the label (most likely class) on [0, 1].
dwRgb = dwImage.select('label').visualize(min=0, max=8, palette=VIS_PALETTE).divide(255)

# Get the most likely class probability.
top1Prob = dwImage.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1].
top1ProbHillshade = ee.Terrain.hillshade(top1Prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dwRgbHillshade = dwRgb.multiply(top1ProbHillshade)

# Define the add_ee_layer function to add Earth Engine layers to folium.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the custom method to the folium Map class.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map.
Map = folium.Map(location=[37.5, 14.0], zoom_start=8)

# Add the Sentinel-2 image layer.
s2_vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}
Map.add_ee_layer(s2Image, s2_vis_params, 'Sentinel-2 L1C')

# Add the Dynamic World visualization layer.
dw_vis_params = {
    'min': 0,
    'max': 0.65
}
Map.add_ee_layer(dwRgbHillshade, dw_vis_params, 'Dynamic World V1 - label hillshade')

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Display the map.
Map


In [5]:
# Create geometries
point = ee.Geometry.Point([1.5, 1.5])
lineString = ee.Geometry.LineString([[-35, -10], [35, -10], [35, 10], [-35, 10]])
linearRing = ee.Geometry.LinearRing(
    [[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]
)
rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20])
polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])


In [6]:
# Create a folium map.
Map = folium.Map(location=[37.5, 14.0], zoom_start=8)

# Add the Sentinel-2 image layer.
s2_vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}
Map.add_ee_layer(s2Image, s2_vis_params, 'Sentinel-2 L1C')

# Add the Dynamic World visualization layer.
dw_vis_params = {
    'min': 0,
    'max': 0.65
}
Map.add_ee_layer(dwRgbHillshade, dw_vis_params, 'Dynamic World V1 - label hillshade')

# Add the geometric layers
Map.add_ee_layer(point, {}, 'Point')
Map.add_ee_layer(lineString, {}, 'LineString')
Map.add_ee_layer(linearRing, {}, 'LinearRing')
Map.add_ee_layer(rectangle, {}, 'Rectangle')
Map.add_ee_layer(polygon, {}, 'Polygon')

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Display the map.
Map


EEException: A mapped function's arguments cannot be used in client-side operations

In [7]:
# Define the start and end date.
START = '2010-04-02'
END = '2018-04-03'

# Define the region of interest.
region = ee.Geometry.Point(14.0, 37.5)

# Filter the collections by region and date.
colFilter = ee.Filter.And(
    ee.Filter.bounds(region),
    ee.Filter.date(START, END))

dwCol = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(colFilter)
s2Col = ee.ImageCollection('COPERNICUS/S2').filter(colFilter)

# Ensure collections are not empty
dwCol_size = dwCol.size().getInfo()
s2Col_size = s2Col.size().getInfo()

if dwCol_size == 0 or s2Col_size == 0:
    raise ValueError("One or both of the collections are empty. Please adjust your filters.")

# Join corresponding DW and S2 images (by system:index).
DwS2Col = ee.Join.saveFirst('s2_img').apply(dwCol, s2Col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'))

# Extract an example DW image and its source S2 image.
dwImage = ee.Image(DwS2Col.first())
s2Image = ee.Image(dwImage.get('s2_img'))


In [8]:
# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']

VIS_PALETTE = [
    '419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b',
    'a59b8f', 'b39fe1']

# Create an RGB image of the label (most likely class) on [0, 1].
dwRgb = dwImage.select('label').visualize(min=0, max=8, palette=VIS_PALETTE).divide(255)

# Get the most likely class probability.
top1Prob = dwImage.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1].
top1ProbHillshade = ee.Terrain.hillshade(top1Prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dwRgbHillshade = dwRgb.multiply(top1ProbHillshade)


In [9]:
# Define the add_ee_layer function to add Earth Engine layers to folium.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the custom method to the folium Map class.
folium.Map.add_ee_layer = add_ee_layer


In [10]:
# Create geometries
point = ee.Geometry.Point([1.5, 1.5])
lineString = ee.Geometry.LineString([[-35, -10], [35, -10], [35, 10], [-35, 10]])
linearRing = ee.Geometry.LinearRing(
    [[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]
)
rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20])
polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])


In [11]:
# Create a folium map.
Map = folium.Map(location=[37.5, 14.0], zoom_start=8)

# Add the Sentinel-2 image layer.
s2_vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}
Map.add_ee_layer(s2Image, s2_vis_params, 'Sentinel-2 L1C')

# Add the Dynamic World visualization layer.
dw_vis_params = {
    'min': 0,
    'max': 0.65
}
Map.add_ee_layer(dwRgbHillshade, dw_vis_params, 'Dynamic World V1 - label hillshade')

# Add the geometric layers
Map.add_ee_layer(point, {}, 'Point')
Map.add_ee_layer(lineString, {}, 'LineString')
Map.add_ee_layer(linearRing, {}, 'LinearRing')
Map.add_ee_layer(rectangle, {}, 'Rectangle')
Map.add_ee_layer(polygon, {}, 'Polygon')

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Display the map.
Map


EEException: A mapped function's arguments cannot be used in client-side operations

In [12]:
# Step 1: Install and Import Required Libraries
!pip install geemap earthengine-api folium

# Step 2: Initialize Earth Engine
import ee
import geemap

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize()

# Step 3: Define the Sicily Region
# Define the Sicily region (bounding box)
sicily_region = ee.Geometry.Polygon([
    [[12.3, 36.4], [12.3, 38.7], [15.7, 38.7], [15.7, 36.4], [12.3, 36.4]]
])

# Step 4: Filter and Select the CORINE Dataset
# Load the CORINE land cover dataset
dataset = ee.Image('COPERNICUS/CORINE/V20/100m/2012')
land_cover = dataset.select('landcover')

# Clip the dataset to the Sicily region
sicily_land_cover = land_cover.clip(sicily_region)

# Step 5: Create and Display the Map
# Create a geemap map centered over Sicily
Map = geemap.Map(center=[37.6, 13.6], zoom=8)

# Add the clipped land cover dataset to the map
Map.addLayer(sicily_land_cover, {}, 'Land Cover (Sicily)')

# Display the map
Map


Map(center=[37.6, 13.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

No such comm: 959840d19be84490b53147074f745f4e


In [13]:
Map.user_roi_coords()

[11.6125, 36.3328, 16.5674, 38.7627]

In [14]:
roi = ee.Geometry.Rectangle([11.6125, 36.3328, 16.5674, 38.7627], None, False)

In [15]:
Map.addLayer(roi, {}, 'sicily_region')

In [16]:
# Define a legend dictionary
legend_dict = {
    'Forest': '397d49',
    'Shrubland': 'dfc35a',
    'Grassland': '88b053',
    'Wetlands': '7a87c6',
    'Water Bodies': '419bdf',
    'Artificial Surfaces': 'c4281b',
    'Bare Areas': 'a59b8f',
    'Permanent Crops': 'e49635',
    'Pastures': '88b053'
}

# Add legend to the map
Map.add_legend(title='CORINE Land Cover Legend', legend_dict=legend_dict)